# CNN to dog breeds

In this notebook, I aim to apply Tensorflow and Pytorch AI frameworks to classify different Dog breed given an image.

## Preparation

In [5]:
# Import libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import glob

## Data

In [36]:
# Load the dog images (.jpg extension)
dog_images = glob.glob("dogimages/**/*.jpg", recursive=True)
num_dog_images = len(dog_images)
print(f"There are {num_dog_images} dog images.")

There are 8351 dog images.


In [ ]:
# Train
train_dog_images = glob.glob("dogimages/train/**/*.jpg", recursive=True)
num_train_dog_images = len(train_dog_images)
print(f"There are {num_train_dog_images} dog images for train.")

There are 6680 dog images for train.


In [ ]:
# Validation
val_dog_images = glob.glob("dogimages/valid/**/*.jpg", recursive=True)
num_val_dog_images = len(val_dog_images)
print(f"There are {num_val_dog_images} dog images for validation.")

There are 835 dog images for validation.


In [ ]:
# Test
test_dog_images = glob.glob("dogimages/test/**/*.jpg", recursive=True)
num_test_dog_images = len(test_dog_images)
print(f"There are {num_test_dog_images} dog images for test.")

There are 836 dog images for test.


In [42]:
# We check that there is no any wrong or missing image
assert num_dog_images == num_train_dog_images + num_val_dog_images + num_test_dog_images

In [22]:
# Different breeds
dog_breeds = glob.glob("dogimages/train/*")
dog_breeds = [dog_breed_tmp.split(".")[1] for dog_breed_tmp in dog_breeds]

## Tensorflow

## Pytorch